The data set encompasses 39,953 locations for 9,924 disasters that occurred worldwide in the years 1960 to 2018. All floods, storms (typhoons, monsoons etc.), earthquakes, landslides, droughts, volcanic activity and extreme temperatures that were recorded in EM-DAT during these 58 years and could be geocoded are included in the data set. The highest spatial resolution in the data set corresponds to administrative level 3 (usually district/commune/village) in the Global Administrative Areas database (GADM, 2018). The vast majority of the locations are administrative level 1 (typically state/province/region).

# !pip install jupyter_bokeh
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import scipy.stats as st
from sklearn.linear_model import LinearRegression

csv_file = 'Resources/pend-gdis-1960-2018-disasterlocations.csv'

# Andy
df = pd.read_csv(csv_file)
df.head()

In [ ]:
# Riley
# Creation of clean data frame

clean_df = df[['id','country','year','level','location','disastertype','disasterno','latitude','longitude']]
clean_df.head()

In [ ]:
# Riley
disaster_count_df = clean_df.drop_duplicates(subset = ['id']).groupby(['level'])['disastertype'].count()
disaster_count_df.head()

In [ ]:
# Riley

# Creation of bar graph showcasing different levels
disaster_count_df.plot(kind = 'bar', title = 'Natural Disasters by Level from 1960 to 2018', xlabel = 'Level', ylabel = 'Natural Disasters',color = ['red','blue','green'])
plt.xticks(rotation='horizontal')
plt.show()



In [ ]:
# Andy
decades = df

bins = [1960,1970,1980,1990,2000,2010,2020]
group_names = ['1960s','1970s','1980s','1990s','2000s','2010s']

decades['decade'] = pd.cut(df['year'],
                           bins, labels=group_names, 
                           include_lowest=True)
decades.head()


In [ ]:
# Riley
def geo_plot_by_decade2(df):
    #points_to_chart = df.loc[df['disastertype'] == disastertype]
    # Create the map plot
    map_plot = df.hvplot.points(
        "longitude",
        "latitude",
        # Configure the map plot
        geo="True", # show geo data
        tiles="OSM",  # OpenStreetMap tiles
        frame_width=700,
        frame_height=500,
        size='level',  # Size of the points determined by Humidity
        color="decade",  # Color by city
        scale=1,
        xlim=(-180, 180),  
        ylim=(-75, 75),
        groupby=['decade', 'disastertype']
    )

    # Display the map
    return map_plot

In [ ]:
# Riley
geo_plot_by_decade2(decades)